In [ ]:
import ase.io as ase_io
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from matplotlib.lines import Line2D

In [ ]:
def get_rmse(a, b, perc=False):
    rmse = np.sqrt(np.mean(np.square(a - b)))
    if perc:
        return 100 * rmse / b.std(ddof=1)
    return rmse

In [ ]:
#  define colors

c0 = "#f18f01"
c1 = "#033f63"
c1bis = "#39b1f9"
c2 = "#95b46a"
c3 = "#ee4266"

In [ ]:
fullerenes = ase_io.read("../../data/fullerenes/fullerenes_resub.xyz", ":")

In [ ]:
natoms = np.asarray([len(frm) for frm in fullerenes])

In [ ]:
castep_energy = np.asarray([frm.info["castep_energy"] for frm in fullerenes])
gomace_energy = np.asarray([frm.info["gomace_energy"] for frm in fullerenes])

psi4_energy = np.asarray([frm.info["spice_energy"] for frm in fullerenes])
maceoff23_energy = np.asarray([frm.info["maceoff23_energy"] for frm in fullerenes])
maceoff24_energy = np.asarray([frm.info["maceoff24_energy"] for frm in fullerenes])

In [ ]:
castep_forces = [x.arrays["castep_forces"] for x in fullerenes]
gomace_forces = [x.arrays["gomace_forces"] for x in fullerenes]

maceoff23_forces = [x.arrays["maceoff23_forces"] for x in fullerenes]
maceoff24_forces = [x.arrays["maceoff24_forces"] for x in fullerenes]
psi4_forces = [x.arrays["spice_forces"] for x in fullerenes]

fullerene with index 16 (0 based) is C$_{\mathrm{60}}$. We use it as energy reference for the uppper panel of our figure


In [ ]:
fig = plt.figure(figsize=(3.5, 3.5), constrained_layout=True)

gs = GridSpec(nrows=3, ncols=1, figure=fig)

ax = fig.add_subplot(gs[:2])

ax.plot(
    natoms,
    gomace_energy / natoms - (gomace_energy / natoms)[16],
    ls="-",
    label="GO-MACE-23",
    lw=1.0,
    marker=".",
    color=c3,
    zorder=10,
    ms=8,
)
ax.plot(
    natoms,
    castep_energy / natoms - (castep_energy / natoms)[16],
    ls="--",
    label="QM",
    lw=0.75,
    c=c3,
)
ax.set_xticklabels((), minor=False)
ax.set_ylabel("Energy (eV at.$^{-1}$)", fontsize=8)
ax.tick_params(axis="both", labelsize=8)

labelx = -0.12
ax.yaxis.set_label_coords(labelx, 0.5, transform=ax.transAxes)

ax.plot(
    natoms,
    maceoff23_energy / natoms - (maceoff23_energy / natoms)[16],
    color=c1,
    marker="d",
    lw=0.75,
    ms=4,
)
ax.plot(
    natoms,
    maceoff24_energy / natoms - (maceoff24_energy / natoms)[16],
    color=c1bis,
    marker="d",
    lw=0.75,
    ms=4,
    ls="-",
    mfc="w",
)
# ax2.plot(
ax.plot(
    natoms,
    psi4_energy / natoms - (psi4_energy / natoms)[16],
    color=c1,
    # marker="d",
    lw=0.75,
    ls="--",
    ms=4,
)


labels = ["ML", "QM", "GO-MACE-23", "MACE-OFF23", "MACE-OFF24"]
custom_lines = [
    Line2D([0], [0], color="k", lw=0.75, ls="-", marker=".", ms=0),
    Line2D([0], [0], color="k", lw=0.75, ls="--", marker=".", ms=0),
    Line2D([0], [0], color=c3, lw=0, ls="-", marker=".", ms=8),
    Line2D([0], [0], color=c1, lw=0, ls="-", marker="d", ms=4),
    Line2D([0], [0], color=c1bis, lw=0, ls="-", marker="d", ms=4, mfc="w"),
]
ax.legend(
    custom_lines,
    labels,
    fontsize=8,
    ncols=3,
    loc="upper right",
    bbox_to_anchor=(0.99, 1.3),
    columnspacing=0.8,
    frameon=False,
)

ax = fig.add_subplot(gs[-1])
ax.plot(
    natoms,
    np.abs(1e3 * (gomace_energy - castep_energy) / natoms),
    ".-",
    lw=1.0,
    label="difference",
    color=c3,
    zorder=10,
    ms=8,
)
ax.plot(
    natoms,
    np.abs(1e3 * (maceoff23_energy - psi4_energy) / natoms),
    "-",
    lw=0.75,
    label="difference",
    color=c1,
    marker="d",
    ms="4",
)
ax.plot(
    natoms,
    np.abs(1e3 * (maceoff24_energy - psi4_energy) / natoms),
    "-",
    lw=0.75,
    label="difference",
    color=c1bis,
    marker="d",
    markerfacecolor="w",
    ms="4",
)
ax.set_yscale("log")
ax.set_ylabel("Energy (meV at.$^{-1}$)", fontsize=8)
ax.tick_params(axis="both", labelsize=8)
ax.set_xlabel("fullerene size", fontsize=8)
ax.yaxis.set_label_coords(labelx, 0.5, transform=ax.transAxes)

# fig.savefig("./fig5.svg", dpi=300, bbox_inches="tight")

In [ ]:
fig = plt.figure(figsize=(3.5, 3.0), constrained_layout=True)

ax = fig.add_subplot(211)

ax.plot(
    natoms,
    np.abs(1e3 * (gomace_energy - castep_energy) / natoms),
    ".-",
    lw=1.0,
    label="difference",
    color=c3,
    zorder=10,
    ms=8,
)
ax.plot(
    natoms,
    np.abs(1e3 * (maceoff23_energy - psi4_energy) / natoms),
    "-",
    lw=0.75,
    label="difference",
    color=c1,
    marker="d",
    ms="4",
)
ax.plot(
    natoms,
    np.abs(1e3 * (maceoff24_energy - psi4_energy) / natoms),
    "-",
    lw=0.75,
    label="difference",
    color=c1bis,
    marker="d",
    markerfacecolor="w",
    ms="4",
)
ax.set_yscale("log")
ax.set_ylabel("Energy AE (meV at.$^{-1}$)", fontsize=8)
# ax.set_xticks(np.arange(3))
ax.set_xticklabels([])
ax.tick_params(axis="both", labelsize=8)


labels = ["GO-MACE-23", "MACE-OFF23", "MACE-OFF24"]
custom_lines = [
    Line2D([0], [0], color=c3, lw=0, ls="-", marker=".", ms=8),
    Line2D([0], [0], color=c1, lw=0, ls="-", marker="d", ms=4),
    Line2D([0], [0], color=c1bis, lw=0, ls="-", marker="d", ms=4, mfc="w"),
]
ax.legend(
    custom_lines,
    labels,
    fontsize=8,
    ncols=3,
    loc="upper right",
    bbox_to_anchor=(0.99, 1.3),
    columnspacing=0.8,
    frameon=False,
)

ax = fig.add_subplot(212)

a = [get_rmse(pred, ref) for pred, ref in zip(gomace_forces, castep_forces)]
ax.plot(
    natoms,
    a,
    ".-",
    lw=1.0,
    label="difference",
    color=c3,
    zorder=10,
    ms=8,
)

a = [get_rmse(pred, ref) for pred, ref in zip(maceoff23_forces, psi4_forces)]
ax.plot(
    natoms,
    a,
    "-",
    lw=0.75,
    label="difference",
    color=c1,
    marker="d",
    ms="4",
)

a = [get_rmse(pred, ref) for pred, ref in zip(maceoff24_forces, psi4_forces)]
ax.plot(
    natoms,
    a,
    "-",
    lw=0.75,
    label="difference",
    color=c1bis,
    marker="d",
    markerfacecolor="w",
    ms="4",
)

ax.set_yscale("log")
ax.set_ylabel("Force RMSE (eV $\\mathrm{\AA}^{-1}$)", fontsize=8)
ax.tick_params(axis="both", labelsize=8)
# ax.minorticks_off()
ax.set_xlabel("fullerene size", fontsize=8)


# fig.savefig("./fullerenes_energy_froce_rmse.svg", dpi=300, bbox_inches="tight")